# 📚 Anki 단어장 생성 프롬프트

## 🔧 시스템 지시사항
너는 영어 텍스트에서 학습 가치가 있는 주요 어휘를 추출하여 Anki 카드용 단어장을 생성하는 AI야.

### 반드시 지켜야 할 규칙
- **너무 쉬운 단어는 제외할 것.**
- **고유명사, 전문용어, 관용구도 포함할 것.**
- **각 카드에는 다음 4개 항목이 반드시 포함될 것:**
  1. 단어 또는 어구
  2. 뜻 (영어 또는 한글)
  3. 품사 (noun, verb, adjective 등)
  4. 영어 예문

- **출력 형식**
  - 각 항목은 세미콜론(;)으로 구분
  - 한 줄에 한 카드
  - 부가설명, 줄바꿈, 추가 문장 모두 금지
  - 결과는 Anki에 바로 업로드할 수 있어야 함

---

## ✅ 입력 텍스트
{사용자가 입력한 영어 텍스트}

---

## ✅ 출력 예시
convene;to come together for a meeting;verb;The committee will convene next week.  
consensus;general agreement;noun;The group reached a consensus after a long discussion.  
proposal;a suggestion or plan;noun;He submitted a proposal for the new project.

---

## 📢 최종 지시
위와 같은 형식으로 **너가 추출한 주요 단어 및 어구를 Anki 카드 포맷으로만 출력하라.**
설명, 주의사항, 줄바꿈 없이 오직 카드 데이터만 출력하라.


In [1]:
from google.colab import userdata
from openai import OpenAI

OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
client = OpenAI(api_key=OPENAI_API_KEY)

In [7]:
from openai import OpenAI
import pandas as pd

# Anki 카드 생성 함수
def generate_anki_cards(en_text, model='gpt-4o-mini', temperature=1, top_p=1):
    system_message = """
너는 영어 텍스트에서 학습 가치가 있는 주요 어휘를 추출하여 Anki 카드용 단어장을 생성하는 AI야.

### 반드시 지켜야 할 규칙
- 너무 쉬운 단어는 제외할 것.
- 고유명사, 전문용어, 관용구도 반드시 포함할 것.

- 각 카드에는 아래 4개 항목이 반드시 포함될 것:
  1. 단어 또는 어구
  2. 뜻 (영어 또는 한글, 자유롭게)
  3. 품사 (noun, verb, adjective 등)
  4. 영어 예문 (반드시 해당 단어를 포함할 것)

### 출력 형식
- 각 항목은 세미콜론(;)으로 구분
- 한 줄에 한 카드
- 부가설명, 줄바꿈, 추가 문장 모두 금지
- 출력은 Anki에 바로 업로드할 수 있어야 함
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": en_text}
        ],
        temperature=temperature,
        max_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content


# Anki 카드 → DataFrame 직접 파싱
def anki_cards_to_dataframe(anki_text):
    lines = anki_text.strip().split("\n")
    data = {'Word/Phrase': [], 'Definition': [], 'Pos': [], 'Example Sentence': []}

    for line in lines:
        parts = line.strip().split(";")
        if len(parts) == 4:
            data['Word/Phrase'].append(parts[0].strip())
            data['Definition'].append(parts[1].strip())
            data['Pos'].append(parts[2].strip())
            data['Example Sentence'].append(parts[3].strip())

    df = pd.DataFrame(data)
    return df
print(generate_anki_cards(en_text))


perfect; 완벽한; adjective; The weather was perfect for a picnic in the park.


In [8]:

# 사용 예시
en_text = 'The weather was perfect for a picnic in the park.'

anki_output = generate_anki_cards(en_text)
print("[Anki 카드 출력]")
print(anki_output)

# 카드 → DataFrame 변환
df = anki_cards_to_dataframe(anki_output)
print("\n[DataFrame]")
print(df)
type(df)


[Anki 카드 출력]
perfect; 완벽한; adjective; The weather was perfect for a picnic in the park.

[DataFrame]
  Word/Phrase Definition        Pos  \
0     perfect        완벽한  adjective   

                                    Example Sentence  
0  The weather was perfect for a picnic in the park.  


pandas.core.frame.DataFrame

## 한번에 할수있는지 실험

In [14]:
def generate_anki_dataframe_code(en_text, model='gpt-4o-mini', temperature=1, top_p=1):
    system_message = """
너는 영어 텍스트에서 학습 가치가 있는 주요 어휘를 추출한 뒤, 해당 어휘들을 Anki 카드 포맷으로 정리하고,
그 데이터를 파싱하여 pandas DataFrame으로 만들기 위한 Python 코드를 생성하는 AI야.

### 반드시 따를 규칙:
1. 먼저 Anki 카드 형식으로 다음과 같이 출력:
   - 단어 또는 어구;뜻;품사;영어 예문
   - 각 항목은 세미콜론(;)으로 구분, 한 줄에 한 카드4

2. 그 다음, 해당 내용을 pandas DataFrame으로 변환하는 Python 코드만 출력할 것
   - 출력 형태는 `df = pd.DataFrame(...)`의 코드
   - columns는 'Word/Phrase', 'Definition', 'Pos', 'Example Sentence'

3. 주석, 설명, 줄바꿈 없이 반드시 코드만 출력할 것
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": en_text}
        ],
        temperature=temperature,
        max_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content

en_text = "The weather was perfect for a picnic in the park"
df = generate_anki_dataframe_code(en_text)
type(df)

str